In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekend.py

(56, 140)
63360
(24, 2637)
[[ 1983.22633887  2024.50886736]
 [ 2334.35986624  2339.7052983 ]
 [ 2559.96474129  2552.65889253]
 [ 3547.48136008  3684.8420239 ]
 [ 1963.04967542  2052.19049633]
 [ 4576.33178681  3802.3362075 ]
 [ 2082.84937986  2128.97412733]
 [ 2113.66448385  2163.78346531]
 [ 2807.29314199  2776.41092719]
 [ 4171.87669634  4591.17268343]
 [ 1970.96490379  1930.37415131]
 [ 3309.98915925  3343.88547788]
 [ 2387.25252558  2562.5324329 ]
 [ 1921.50484729  2003.61446613]
 [ 1646.75562958  1645.98058364]
 [ 1350.38029894  1298.94593462]
 [ 1407.5342739   1407.5342739 ]
 [ 2522.50225242  2544.44850137]
 [ 2417.69939095  2417.69939095]
 [ 2826.80230034  2838.534693  ]
 [ 1819.10224428  1866.25828208]
 [ 2051.4420253   2198.36449548]
 [ 3516.68285291  3516.68285291]
 [ 1908.0771636   1998.53995697]]
(24, 140)
Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective ra